# SimCLR Code with Tensorflow

In [1]:
import tensorflow as tf
print(tf.__version__)

2.17.1


## Environment Setup

In [ ]:
#!sudo apt-get update
#!sudo apt-get install build-essential
#!sudo apt-get install python3-dev

In [ ]:
!pip install tf-models-official

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_models as tfm
import tensorflow_datasets as tfds
import tarfile
import matplotlib.pyplot as plt
from PIL import Image
from utils.NT_Xent import NT_Xent_tf
from utils.evaluation_metrics import get_top_k_accuracy
from training.learning_rate_schedule import MyLRSchedule
from training.train import train_step, train
from utils.test import test
from utils.data_augmentations import augment_image
import seaborn as sns
import os

## Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tar_file_path = '/content/drive/MyDrive/imagenet-lsvrc-2012/ILSVRC2012_img_train_t3.tar'
extract_path = '/content/imagenet_data'

# Extract the tar file
with tarfile.open(tar_file_path, 'r') as tar:
    tar.extractall(path=extract_path)

print(f"Extracted files to: {extract_path}")

Extracted files to: /content/imagenet_data


In [ ]:
imagenet_tar_path = '/content/imagenet_data'
output_path = '/content/organized_imagenet_data'
os.makedirs(output_path, exist_ok=True)

class_count = 0
class_limit = 5

# Loop through all .tar files
for tar_file in os.listdir(imagenet_tar_path):
    if tar_file.endswith('.tar'):
        class_name = tar_file.split('.tar')[0]  # Extract class name (e.g., n02085620)
        class_output_dir = os.path.join(output_path, class_name)

        # Make directory for this class
        os.makedirs(class_output_dir, exist_ok=True)

        # Extract .tar file into the class directory
        tar_file_path = os.path.join(imagenet_tar_path, tar_file)
        with tarfile.open(tar_file_path, 'r') as tar:
            tar.extractall(path=class_output_dir)

        print(f"Extracted {tar_file} into {class_output_dir}")

    class_count += 1
    if class_count > class_limit:
        break

print("Extraction and organization complete.")

Extracted n02094258.tar into /content/organized_imagenet_data/n02094258
Extracted n02100583.tar into /content/organized_imagenet_data/n02100583
Extracted n02086240.tar into /content/organized_imagenet_data/n02086240
Extracted n02105162.tar into /content/organized_imagenet_data/n02105162
Extracted n02109047.tar into /content/organized_imagenet_data/n02109047
Extracted n02093647.tar into /content/organized_imagenet_data/n02093647
Extraction and organization complete.


In [ ]:
picture_limit = 10000000
output_path = '/content/organized_imagenet_data'

for class_name in os.listdir(output_path):
    count = 0
    for image_name in os.listdir(os.path.join(output_path, class_name)):
        count += 1
        if count > picture_limit:
            del_path = os.path.join(output_path, class_name, image_name)
            os.remove(del_path)

In [ ]:
# Parameters
batch_size = 16
img_height = 224
img_width = 224

# Load dataset
dataset = tf.keras.utils.image_dataset_from_directory(
    output_path,
    labels='inferred',  # Automatically infer labels from subfolder names
    label_mode='int',   # Labels are integers
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True
)

# Print class names
class_names = dataset.class_names
print("Classes:", class_names)

# Inspect a batch of images
for images, labels in dataset.take(1):
    print("Image shape:", images.shape)
    print("Labels:", labels)

Found 1041 files belonging to 6 classes.
Classes: ['n02086240', 'n02093647', 'n02094258', 'n02100583', 'n02105162', 'n02109047']
Image shape: (16, 224, 224, 3)
Labels: tf.Tensor([0 5 4 1 0 1 4 3 5 1 2 2 2 2 5 2], shape=(16,), dtype=int32)


In [ ]:
# Calculate the sizes of each split
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size  # Ensure full split

# Unbatch the dataset
full_dataset = dataset.unbatch()

# Create the training, validation, and testing datasets
train_dataset = full_dataset.take(train_size).batch(batch_size)
val_dataset = full_dataset.skip(train_size).take(val_size).batch(batch_size)
test_dataset = full_dataset.skip(train_size + val_size).take(test_size).batch(batch_size)

# Print the sizes of the resulting datasets
train_dataset_count = sum(1 for _ in train_dataset)
val_dataset_count = sum(1 for _ in val_dataset)
test_dataset_count = sum(1 for _ in test_dataset)

print(f"Training dataset size: {train_dataset_count} batches")
print(f"Validation dataset size: {val_dataset_count} batches")
print(f"Testing dataset size: {test_dataset_count} batches")

Training dataset size: 4 batches
Validation dataset size: 1 batches
Testing dataset size: 1 batches


## Pretraining ResNet-50 with Linear and Non-linear Projection Heads

### non-linear proejction head

In [ ]:
# define the necessary variables
epochs = 100
temperature = 0.5
base_learning_rate = 0.03 * batch_size / 256
warmup_epochs = 10

# Create the learning rate schedule
lr_schedule = MyLRSchedule(base_learning_rate, warmup_epochs * train_dataset_count, epochs * train_dataset_count)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.SGD(
    learning_rate=lr_schedule,
    momentum=0.9,
    nesterov=True,
    weight_decay=None,
)

## define mdoel (backbone + projection head)
backbone = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='avg'
)
backbone.trainable = True # just like this in pretraining

projection_head = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128)
])
projection_head.trainable = True

inputs = tf.keras.layers.Input(shape=(224, 224, 3))
feature_extractor = backbone(inputs)
projection_output = projection_head(feature_extractor)
model = tf.keras.models.Model(inputs=inputs, outputs=projection_output)

# train the model
model, optimizer, losses, top_1_acuracies, top_5_acuracies, test_accuracies = train(epochs, train_dataset, model, optimizer, temperature, print_acc=False)

Epoch 1, Average Loss: 2.7400
Epoch 2, Average Loss: 2.8292
Epoch 3, Average Loss: 2.7059
Epoch 4, Average Loss: 2.6213
Epoch 5, Average Loss: 2.5833
Epoch 6, Average Loss: 2.4885
Epoch 7, Average Loss: 2.3953
Epoch 8, Average Loss: 2.3339
Epoch 9, Average Loss: 2.3082
Epoch 10, Average Loss: 2.2668
Epoch 11, Average Loss: 2.2188
Epoch 12, Average Loss: 2.2321
Epoch 13, Average Loss: 2.1114
Epoch 14, Average Loss: 2.3681
Epoch 15, Average Loss: 2.4951
Epoch 16, Average Loss: 2.8228
Epoch 17, Average Loss: 2.6576
Epoch 18, Average Loss: 2.5739
Epoch 19, Average Loss: 2.9459
Epoch 20, Average Loss: 2.8859
Epoch 21, Average Loss: 2.8566
Epoch 22, Average Loss: 2.7003
Epoch 23, Average Loss: 2.3059
Epoch 24, Average Loss: 2.6411
Epoch 25, Average Loss: 2.4306
Epoch 26, Average Loss: 2.1939
Epoch 27, Average Loss: 2.5035
Epoch 28, Average Loss: 2.3431
Epoch 29, Average Loss: 2.2950
Epoch 30, Average Loss: 2.2808
Epoch 31, Average Loss: 2.2275
Epoch 32, Average Loss: 2.2227
Epoch 33, Average

In [ ]:
model.save('models/pretrained_resnet_nonlinear_head.keras')

In [ ]:
plt.clf()
plt.plot(losses, label='Training Loss')
plt.title('Training Loss for Non-linear Head')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('figures/nonlinear_pretraining_loss.png')

### linear projection head

In [ ]:
# define the necessary variables
epochs = 100
temperature = 0.5
base_learning_rate = 0.03 * batch_size / 256
warmup_epochs = 10

# Create the learning rate schedule
lr_schedule = MyLRSchedule(base_learning_rate, warmup_epochs * train_dataset_count, epochs * train_dataset_count)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.SGD(
    learning_rate=lr_schedule,
    momentum=0.9,
    nesterov=True,
    weight_decay=None,
)

## define mdoel (backbone + projection head)
backbone = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='avg'
)
backbone.trainable = True # just like this in pretraining

projection_head = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128)
])
projection_head.trainable = True

inputs = tf.keras.layers.Input(shape=(224, 224, 3))
feature_extractor = backbone(inputs)
projection_output = projection_head(feature_extractor)
model_nonloienar = tf.keras.models.Model(inputs=inputs, outputs=projection_output)

# train the model
model_nonloienar, optimizer, losses_nonlinear, top_1_acuracies, top_5_acuracies, test_accuracies = train(epochs, train_dataset, model_nonloienar, optimizer, temperature, print_acc=False)

Epoch 1, Average Loss: 2.6166
Epoch 2, Average Loss: 2.6955
Epoch 3, Average Loss: 2.6085
Epoch 4, Average Loss: 2.5898
Epoch 5, Average Loss: 2.5270
Epoch 6, Average Loss: 2.3617
Epoch 7, Average Loss: 2.3030
Epoch 8, Average Loss: 2.4121
Epoch 9, Average Loss: 2.4311
Epoch 10, Average Loss: 2.4213
Epoch 11, Average Loss: 2.2522
Epoch 12, Average Loss: 2.2802
Epoch 13, Average Loss: 2.2557
Epoch 14, Average Loss: 2.3357
Epoch 15, Average Loss: 2.2401
Epoch 16, Average Loss: 2.4531
Epoch 17, Average Loss: 2.3928
Epoch 18, Average Loss: 2.3849
Epoch 19, Average Loss: 2.1766
Epoch 20, Average Loss: 2.0193
Epoch 21, Average Loss: 2.1311
Epoch 22, Average Loss: 2.1899
Epoch 23, Average Loss: 2.6304
Epoch 24, Average Loss: 2.6397
Epoch 25, Average Loss: 2.3502
Epoch 26, Average Loss: 2.5126
Epoch 27, Average Loss: 2.2565
Epoch 28, Average Loss: 2.3170
Epoch 29, Average Loss: 2.2543
Epoch 30, Average Loss: 2.1492
Epoch 31, Average Loss: 2.1098
Epoch 32, Average Loss: 2.1086
Epoch 33, Average

In [ ]:
model_nonloienar.save('models/pretrained_resnet_linear_head.keras')

In [ ]:
plt.clf() ## to empty the buffer of plt - it showed more results every time I ran the code
plt.plot(losses_nonlinear, label='Training Loss')
plt.title('Training Loss for Linear Projection Head')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('figures/linear_pretraining_loss.png')

### loss comparison

In [ ]:
plt.clf()
plt.plot(losses, label='Non-linear projection head')
plt.plot(losses_nonlinear, label='Linear projection head')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('figures/loss_comparison.png')

## Linear Classifier Evaluation Setting + Sample Run

In [ ]:
## load saved model from keras
nonlinear_model_path = '/content/drive/MyDrive/ecbm4040-finalproject/models/pretrained_resnet_nonlinear_head.keras'
resnet_pretrained_nonlinear_head = tf.keras.models.load_model(nonlinear_model_path)

epochs = 15
temperature = 0.7
base_learning_rate = 0.1 * batch_size / 256
warmup_epochs = 0
n_classes = 6

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.SGD(
    learning_rate=base_learning_rate,
    momentum=0.9,
    nesterov=True,
    weight_decay=None,
)

# Create the loss function or criterion
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Add linear classifier on top the backbone
linear_classifier = tf.keras.models.Sequential([tf.keras.layers.Dense(n_classes, activation=None)])
linear_classifier.trainable = True

inputs = tf.keras.layers.Input(shape=(224, 224, 3))
feature_extractor = resnet_pretrained_nonlinear_head(inputs)
linear_classifier_output = linear_classifier(feature_extractor)
model = tf.keras.models.Model(inputs=inputs, outputs=linear_classifier_output)

## compile and run model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
model.fit(train_dataset, epochs=epochs)

Epoch 1/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 57s 6s/step - accuracy: 0.2928 - loss: 1.8764
Epoch 2/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6705 - loss: 1.0129
Epoch 3/15


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8272 - loss: 0.4705
Epoch 4/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8210 - loss: 0.5299
Epoch 5/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8982 - loss: 0.4535
Epoch 6/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7973 - loss: 0.7167
Epoch 7/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7806 - loss: 0.6331
Epoch 8/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8216 - loss: 0.7157
Epoch 9/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8899 - loss: 0.4174
Epoch 10/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8079 - loss: 0.6831
Epoch 11/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8127 - loss: 0.7445
Epoch 12/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8614 - loss: 0.4207
Epoch 13/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9018 - loss: 0.5019
Epoch 14/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8196 - loss: 0.5259
Epoch 15/1

In [ ]:
# get loss and accuracy over test dataset
model.evaluate(test_dataset)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.3750 - loss: 6.3851


[6.385066509246826, 0.375]

In [ ]:
plt.clf()
plt.plot(model.history.history['loss'], label='Linear Classifier Loss')
plt.title('Linear Classifier Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('figures/linear_classifier_sample_running_loss.png')

In [ ]:
plt.clf()
plt.plot(model.history.history['accuracy'], label='Linear Classifier Accuracy')
plt.title('Linear Classifier Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('figures/linear_classifier_sample_running_accuracy.png')

In [ ]:
model.save('models/linear_classifier_sample_run.keras')

## Experiment 1: Augmentation Pair testing

Here I run the first experiment for finding the best combo of the data augmentation functions. Since the table in the original paper (figure 5) is not symmetric, I went over the ['aug1', 'aug2'] combinations two times

In [ ]:
augmentations = ['random_crop_and_resize', 'horizontal_flip', 'color_distortion', 'gaussian_blur', 'cutout', 'sobel_filter', 'gaussian_noise', 'rotate']

## get all the augmentaion pairs
## since
all_augmentation_pairs = []

for aug in augmentations:
    for aug2 in augmentations:
        all_augmentation_pairs.append((aug, aug2))

In [ ]:
epochs = 5
temperature = 0.7
base_learning_rate = 0.1 * batch_size / 256
warmup_epochs = 0
n_classes = 6
nonlinear_model_path = '/content/drive/MyDrive/ecbm4040-finalproject/models/pretrained_resnet_nonlinear_head.keras'

results = {pair_of_augmentations: [0.0, 0.0] for pair_of_augmentations in all_augmentation_pairs}

for augmentation_pair in all_augmentation_pairs:
    print(f"Augmentation pair: {augmentation_pair}")

    ### linear classifier evaluation here

    optimizer = tf.keras.optimizers.SGD(learning_rate=base_learning_rate, momentum=0.9, nesterov=True, weight_decay=None)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    resnet_pretrained_nonlinear_head = tf.keras.models.load_model(nonlinear_model_path)
    resnet_pretrained_nonlinear_head.trainable = False

    linear_classifier = tf.keras.models.Sequential([tf.keras.layers.Dense(n_classes, activation=None)])
    linear_classifier.trainable = True

    inputs = tf.keras.layers.Input(shape=(224, 224, 3))
    feature_extractor = resnet_pretrained_nonlinear_head(inputs)
    linear_classifier_output = linear_classifier(feature_extractor)
    model = tf.keras.models.Model(inputs=inputs, outputs=linear_classifier_output)

    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
    model.fit(train_dataset, epochs=epochs, verbose=0)

    results[augmentation_pair][0] = model.history.history['accuracy'][-1] #get train accuracy
    results[augmentation_pair][1] = model.evaluate(test_dataset, verbose=0)[1] #get test accuracy

Augmentation pair: ('random_crop_and_resize', 'random_crop_and_resize')
Augmentation pair: ('random_crop_and_resize', 'horizontal_flip')
Augmentation pair: ('random_crop_and_resize', 'color_distortion')
Augmentation pair: ('random_crop_and_resize', 'gaussian_blur')
Augmentation pair: ('random_crop_and_resize', 'cutout')
Augmentation pair: ('random_crop_and_resize', 'sobel_filter')
Augmentation pair: ('random_crop_and_resize', 'gaussian_noise')
Augmentation pair: ('random_crop_and_resize', 'rotate')
Augmentation pair: ('horizontal_flip', 'random_crop_and_resize')
Augmentation pair: ('horizontal_flip', 'horizontal_flip')
Augmentation pair: ('horizontal_flip', 'color_distortion')
Augmentation pair: ('horizontal_flip', 'gaussian_blur')
Augmentation pair: ('horizontal_flip', 'cutout')
Augmentation pair: ('horizontal_flip', 'sobel_filter')
Augmentation pair: ('horizontal_flip', 'gaussian_noise')
Augmentation pair: ('horizontal_flip', 'rotate')
Augmentation pair: ('color_distortion', 'random_

In [ ]:
train_results_table = []
test_results_table = []

for aug1 in augmentations:

    train_resulsts = []
    test_results = []

    for aug2 in augmentations:
        train_resulsts.append(results[(aug1, aug2)][0])
        test_results.append(results[(aug1, aug2)][1])

    train_results_table.append(train_resulsts)
    test_results_table.append(test_results)

In [ ]:
# creating sns heatmap and save
plt.clf()
sns.set(font_scale=1)
fig, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(train_results_table, annot=True, cmap='Blues', ax=ax)
plt.xticks(np.arange(len(augmentations)) + 0.25, augmentations, rotation=45)
plt.yticks(np.arange(len(augmentations)) + 0.25, augmentations, rotation=0)
plt.xlabel('Augmentation 2')
plt.ylabel('Augmentation 1')
plt.title('Train Accuracy Heatmap')
plt.savefig('figures/train_accuracy_heatmap.png')

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(test_results_table, annot=True, cmap='Blues', ax=ax)
plt.xticks(np.arange(len(augmentations)) + 0.25, augmentations, rotation=45)
plt.yticks(np.arange(len(augmentations)) + 0.25, augmentations, rotation=0)
plt.xlabel('Augmentation 2')
plt.ylabel('Augmentation 1')
plt.title('Test Accuracy Heatmap')
plt.savefig('figures/test_accuracy_heatmap.png')

## Experiment 2: Comparing linear vs non-linear projection heads

In [ ]:
######## Pretraining ResNet-50 with Non-linear and Linear Projection Heads ########

# define the necessary variables
epochs = 15
temperature = 0.5
base_learning_rate = 0.1 * batch_size / 256
warmup_epochs = 0
n_classes = 6
hidden_dims = [2**i for i in range(5, 12)]

# Create the learning rate schedule
lr_schedule = MyLRSchedule(base_learning_rate, warmup_epochs * train_dataset_count, epochs * train_dataset_count)

linear_head_accuracies = []
nonlinear_head_accuracies = []

for hidden_dim in hidden_dims:

    for model_type in ['nonlinear', 'linear']:

        ##### training part #####
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True, weight_decay=None,)

        backbone = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3), pooling='avg')
        backbone.trainable = True

        # create the approporiate projection head
        projection_head = tf.keras.models.Sequential()
        if model_type == 'nonlinear':
            projection_head.add(tf.keras.layers.Dense(512, activation='relu'))
        projection_head.add(tf.keras.layers.Dense(hidden_dim))
        projection_head.trainable = True

        # add projection head and backbone together
        inputs = tf.keras.layers.Input(shape=(224, 224, 3))
        feature_extractor = backbone(inputs)
        projection_output = projection_head(feature_extractor)
        model = tf.keras.models.Model(inputs=inputs, outputs=projection_output)

        # train the model
        model, optimizer, losses, top_1_acuracies, top_5_acuracie, test_acc = train(epochs, train_dataset, model, optimizer, temperature, print_acc=False)

        ##### evaluation part #####
        optimizer = tf.keras.optimizers.SGD(learning_rate=base_learning_rate,momentum=0.9,nesterov=True,weight_decay=None,)
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

        model.trainable = False
        linear_classifier = tf.keras.models.Sequential([tf.keras.layers.Dense(n_classes, activation=None)])
        linear_classifier.trainable = True

        inputs = tf.keras.layers.Input(shape=(224, 224, 3))
        feature_extractor = model(inputs)
        linear_classifier_output = linear_classifier(feature_extractor)
        evaluation_model = tf.keras.models.Model(inputs=inputs, outputs=linear_classifier_output)

        ## compile and run model
        evaluation_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
        evaluation_model.fit(train_dataset, epochs=epochs, verbose=0)

        if model_type == 'nonlinear':
            nonlinear_head_accuracies.append([evaluation_model.history.history['accuracy'][-1], evaluation_model.evaluate(test_dataset, verbose=0)[1]])
        else:
            linear_head_accuracies.append([evaluation_model.history.history['accuracy'][-1], evaluation_model.evaluate(test_dataset, verbose=0)[1]])

Epoch 1, Average Loss: 2.8488
Epoch 2, Average Loss: 2.9312
Epoch 3, Average Loss: 3.0591
Epoch 4, Average Loss: 3.0616
Epoch 5, Average Loss: 3.0618
Epoch 6, Average Loss: 3.0619
Epoch 7, Average Loss: 3.0617
Epoch 8, Average Loss: 3.0617
Epoch 9, Average Loss: 3.0618
Epoch 10, Average Loss: 3.0618
Epoch 11, Average Loss: 3.0616
Epoch 12, Average Loss: 3.0618
Epoch 13, Average Loss: 3.0616
Epoch 14, Average Loss: 3.0619
Epoch 15, Average Loss: 3.0616


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Epoch 1, Average Loss: 2.8365
Epoch 2, Average Loss: 3.0538
Epoch 3, Average Loss: 3.0604
Epoch 4, Average Loss: 3.0589
Epoch 5, Average Loss: 3.0574
Epoch 6, Average Loss: 3.0609
Epoch 7, Average Loss: 3.0607
Epoch 8, Average Loss: 3.0607
Epoch 9, Average Loss: 3.0595
Epoch 10, Average Loss: 3.0512
Epoch 11, Average Loss: 3.0404
Epoch 12, Average Loss: 3.0615
Epoch 13, Average Loss: 3.0616
Epoch 14, Average Loss: 3.0613
Epoch 15, Average Loss: 3.0615
Epoch 1, Average Loss: 2.7621
Epoch 2, Average Loss: 2.8443
Epoch 3, Average Loss: 3.0527
Epoch 4, Average Loss: 3.0593
Epoch 5, Average Loss: 3.0578
Epoch 6, Average Loss: 3.0184
Epoch 7, Average Loss: 3.0619
Epoch 8, Average Loss: 3.0620
Epoch 9, Average Loss: 3.0620
Epoch 10, Average Loss: 3.0620
Epoch 11, Average Loss: 3.0620
Epoch 12, Average Loss: 3.0620
Epoch 13, Average Loss: 3.0620
Epoch 14, Average Loss: 3.0620
Epoch 15, Average Loss: 3.0620
Epoch 1, Average Loss: 2.7609
Epoch 2, Average Loss: 2.7867
Epoch 3, Average Loss: 2.799

In [ ]:
######## Pretraining ResNet-50 for Comparison ########

# define the necessary variables
epochs = 15
temperature = 0.5
base_learning_rate = 0.1 * batch_size / 256
warmup_epochs = 0
n_classes = 6

##### training part #####
# Training model with no projection head for comparison

optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True, weight_decay=None,)
backbone = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3), pooling='avg')
backbone.trainable = True

inputs = tf.keras.layers.Input(shape=(224, 224, 3))
outputs = backbone(inputs)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

model, optimizer, losses, top_1_acuracies, top_5_acuracie, test_acc = train(epochs, train_dataset, model, optimizer, temperature, print_acc=False)

##### evaluation part #####
model.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=base_learning_rate,momentum=0.9,nesterov=True,weight_decay=None,)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

backbone = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3), pooling='avg')
backbone.trainable = True

inputs = tf.keras.layers.Input(shape=(224, 224, 3))
outputs = backbone(inputs)
evaluation_model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

evaluation_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
evaluation_model.fit(train_dataset, epochs=epochs, verbose=0)

train_acc_no_projection = evaluation_model.history.history['accuracy'][-1]
test_acc_no_projection = evaluation_model.evaluate(test_dataset, verbose=0)[1]

Epoch 1, Average Loss: 2.6435
Epoch 2, Average Loss: 2.7298
Epoch 3, Average Loss: 2.7555
Epoch 4, Average Loss: 2.7597
Epoch 5, Average Loss: 3.0508
Epoch 6, Average Loss: 3.0594
Epoch 7, Average Loss: 3.0600
Epoch 8, Average Loss: 3.0601
Epoch 9, Average Loss: 3.0569
Epoch 10, Average Loss: 3.0570
Epoch 11, Average Loss: 3.0430
Epoch 12, Average Loss: 3.0349
Epoch 13, Average Loss: 2.9797
Epoch 14, Average Loss: 2.9447
Epoch 15, Average Loss: 2.9388


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [ ]:
bar_width = 0.35
positions = [i for i in range(len(hidden_dims))]
positions_linear = [p - bar_width / 2 for p in positions]
positions_nonlinear = [p + bar_width / 2 for p in positions]

plt.clf()
plt.bar(positions_linear, [e[0] for e in linear_head_accuracies], width=bar_width, label='Linear Head')
plt.bar(positions_nonlinear, [e[0] for e in nonlinear_head_accuracies], width=bar_width, label='Nonelinear Head')
plt.bar(positions_nonlinear[-1] + bar_width, train_acc_no_projection, width=bar_width, label='No Projection Head')
plt.legend()
plt.xticks(positions, hidden_dims)
plt.xlabel('Hidden Dimension')
plt.ylabel('Top-1 Accuracy')
plt.title('Top-1 Train Accuracy vs Hidden Dimension ')
plt.savefig('figures/bar_projection_heads_accuracies_train.png')

In [ ]:
bar_width = 0.35
positions = [i for i in range(len(hidden_dims))]
positions_linear = [p - bar_width / 2 for p in positions]
positions_nonlinear = [p + bar_width / 2 for p in positions]

plt.clf()
plt.bar(positions_linear, [e[1] for e in linear_head_accuracies], width=bar_width, label='Linear Head')
plt.bar(positions_nonlinear, [e[1] for e in nonlinear_head_accuracies], width=bar_width, label='Nonelinear Head')
plt.bar(positions_nonlinear[-1] + bar_width, test_acc_no_projection, width=bar_width, label='No Projection Head')
plt.legend()
plt.xticks(positions, hidden_dims)
plt.xlabel('Hidden Dimension')
plt.ylabel('Top-1 Accuracy')
plt.title('Top-1 Test Accuracy vs Hidden Dimension')
plt.savefig('figures/bar_projection_heads_accuracies_test.png')

In [ ]:
augmentations = ['random_crop_and_resize', 'horizontal_flip', 'color_distortion', 'gaussian_blur', 'cutout', 'sobel_filter', 'gaussian_noise', 'rotate']

for images, label in train_dataset.take(1):
    for img in images:
        X_augmented_1 = augment_image(img, target_size=(224, 224), target_augmentations=augmentations)
        plt.imshow(X_augmented_1 / 255)
        plt.show()

In [ ]:
import os
import pathlib

def print_tree(directory, level=0):
    path = pathlib.Path(directory)
    for item in path.iterdir():
        print("|   " * level + "+-- " + item.name)
        if item.is_dir():
            print_tree(item, level + 1)

# Example usage
print_tree('.')


+-- .git
|   +-- branches
|   +-- info
|   |   +-- exclude
|   +-- hooks
|   |   +-- pre-merge-commit.sample
|   |   +-- pre-commit.sample
|   |   +-- prepare-commit-msg.sample
|   |   +-- push-to-checkout.sample
|   |   +-- pre-applypatch.sample
|   |   +-- applypatch-msg.sample
|   |   +-- fsmonitor-watchman.sample
|   |   +-- pre-receive.sample
|   |   +-- post-update.sample
|   |   +-- pre-rebase.sample
|   |   +-- pre-push.sample
|   |   +-- commit-msg.sample
|   |   +-- update.sample
|   +-- description
|   +-- refs
|   |   +-- heads
|   |   |   +-- main
|   |   +-- tags
|   |   +-- remotes
|   |   |   +-- origin
|   |   |   |   +-- HEAD
|   +-- objects
|   |   +-- pack
|   |   |   +-- pack-b896c96a6807667d984bca8e6ced78c99265ce87.pack
|   |   |   +-- pack-b896c96a6807667d984bca8e6ced78c99265ce87.idx
|   |   +-- info
|   +-- config
|   +-- packed-refs
|   +-- logs
|   |   +-- refs
|   |   |   +-- remotes
|   |   |   |   +-- origin
|   |   |   |   |   +-- HEAD
|   |   |   +-- head